In [5]:
import pandas as pd
case_land = pd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\CaseLands_03032025_1330.parquet")
case_land = case_land.drop(columns='Shape')
case_land.to_parquet("CaseLands_03032025_1330.parquet", index=False)

In [6]:
import pandas as pd
import json
import os

schema_xcel = r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\MLRS_Pub_Extract_Schema_20241104.xlsx"
df = pd.read_excel(schema_xcel, sheet_name='Tables', header = None)

# Ensure output directory exists
os.makedirs("schemas", exist_ok=True)

# Initialize
tables = {}
current_table = None

# Loop through spreadsheet rows
for _, row in df.iterrows():
    table_candidate = row[0]
    column_type = row[1] if len(row) > 1 else None

    # If only table name is present (new section)
    if pd.notna(table_candidate) and pd.isna(column_type):
        current_table = table_candidate.strip()
        tables[current_table] = []
    elif current_table and pd.notna(table_candidate) and pd.notna(column_type):
        column_name = table_candidate.strip()
        tables[current_table].append({
            "name": column_name,
            "type": "STRING"
        })

# Write each schema as JSON file
for table, schema in tables.items():
    filename = f"schemas/{table.lower()}_schema.json"
    with open(filename, "w") as f:
        json.dump(schema, f, indent=4)
    print(f"Saved: {filename}")


Saved: schemas/account_rpt_schema.json
Saved: schemas/action_schema.json
Saved: schemas/admin_unit_schema.json
Saved: schemas/admin_unit_agg_schema.json
Saved: schemas/blm_case_schema.json
Saved: schemas/blm_product_schema.json
Saved: schemas/case_action_schema.json
Saved: schemas/case_customer_schema.json
Saved: schemas/case_land_schema.json
Saved: schemas/case_transaction_schema.json
Saved: schemas/intrel_d_schema.json
Saved: schemas/product_fee_schema.json
Saved: schemas/rpt_calendar_schema.json
Saved: schemas/case_action_status_d_schema.json
Saved: schemas/commodity_d_schema.json
Saved: schemas/case_group_d_schema.json
Saved: schemas/meridian_d_schema.json
Saved: schemas/survey_type_d_schema.json
Saved: schemas/action_case_schema.json
Saved: schemas/action_land_schema.json
Saved: schemas/case_admin_unit_schema.json
Saved: schemas/record_type_schema.json
Saved: schemas/supplemental_use_schema.json
Saved: schemas/us_right_case_land_schema.json


In [7]:
import pandas as pd

schema_xcel = r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\MLRS_Pub_Extract_Schema_20241104.xlsx"
df = pd.read_excel(schema_xcel, sheet_name='Tables', header = None)

# Snapshot date
snapshot_date = "20250401"

# GCS and BQ setup
gcs_base = f"gs://sandbox-blm-seta-dqimp-qaqc/snapshots/{snapshot_date}"
bq_dataset = "blm_dqimp_qaqc"

# Extract table names
tables = []
for _, row in df.iterrows():
    table_candidate = row[0]
    column_type = row[1] if len(row) > 1 else None
    if pd.notna(table_candidate) and pd.isna(column_type):
        tables.append(table_candidate.strip())

# Create the output lines
lines = []
for table in tables:
    upper_table = table.upper()
    lower_table = table.lower()

    lines.append(f"combine {upper_table}:")
    lines.append(
        f"gsutil compose {gcs_base}/MC_{upper_table}.csv {gcs_base}/CR_FULL_{upper_table}.csv {gcs_base}/{upper_table}.csv"
    )
    lines.append(
        f"gsutil rm {gcs_base}/MC_{upper_table}.csv {gcs_base}/CR_FULL_{upper_table}.csv\n"
    )
    lines.append(f"Load {upper_table}:")
    lines.append(
        f"bq load --source_format=CSV --field_delimiter=\"|\" --skip_leading_rows=0 --schema={lower_table}_schema.json {bq_dataset}.{lower_table}_{snapshot_date} {gcs_base}/{upper_table}.csv\n"
    )

# Save to file
with open("load_commands.txt", "w") as f:
    f.write("\n".join(lines))

print("✅ File saved: load_commands.txt")


✅ File saved: load_commands.txt
